# Inference examples

In [ ]:
# Do not forget to install all dependencies first:
!pip install -Uqq WhisperSpeech

In [ ]:
def is_colab():
    try: import google.colab; return True
    except: return False

import torch
if not torch.cuda.is_available():
    if is_colab(): raise BaseException("Please change the runtime type to GPU. In the menu: Runtime -> Change runtime type (the free T4 instance is enough)")
    else:          raise BaseException("Currently the example notebook requires CUDA, make sure you are running this on a machine with a GPU.")

In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import torch
import torch.nn.functional as F

from IPython.display import Markdown, HTML

device = "cuda" if torch.cuda.is_available() else "cpu"

## The whole pipeline

In [2]:
from whisperspeech.pipeline import Pipeline
import torchaudio
import nltk
from nltk.tokenize import sent_tokenize
import re

In [3]:
pipe = Pipeline(torch_compile=True)
nltk.download('punkt')

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [20]:
# sample_text = """Electromagnetism is a fundamental force of nature that encompasses the interaction between
# electrically charged particles. It is described by Maxwell's equations, which unify electricity, magnetism,
# and light into a single theory. In essence, electric charges produce electric fields that exert forces on
# other charges, while moving charges (currents) generate magnetic fields. These magnetic fields, in turn,
# can affect the motion of charges and currents. The interaction between electric and magnetic fields propagates
# through space as electromagnetic waves, which include visible light, radio waves, and X-rays. Electromagnetic
# forces are responsible for practically all the phenomena encountered in daily life, excluding gravity.
# """

sample_text = """
It's Coach Bennett here!
Listen up, champions!
Here's your nugget of wisdom for today: Your mind quits a thousand times before your body does.
So when that little voice says 'I'm done,' you look it straight in the eye and say 'We're just getting started!'
Remember, every step is a victory, every mile a triumph.
Your legs might be screaming, but your heart? It's singing!
So lace up tight, stand tall, and show that road who's boss.
You're not just running, you're rewriting your story with every stride.
""".strip()

In [14]:
def split_and_prepare_text(text, cps=14):
    chunks = []
    sentences = sent_tokenize(text)
    chunk = ""
    for sentence in sentences:
        # replace fancy punctuation that was unseen during training
        sentence = re.sub('[()]', ",", sentence).strip()
        sentence = re.sub(",+", ",", sentence)
        sentence = re.sub('"+', "", sentence)
        sentence = re.sub("/", "", sentence)
        # merge until the result is < 20s
        if len(chunk) + len(sentence) < 20*cps:
            chunk += " " + sentence
        else:
            chunks.append(chunk)
            chunk = sentence
    if chunk: chunks.append(chunk)
    return chunks

In [15]:
split_and_prepare_text(sample_text)

[" Electromagnetism is a fundamental force of nature that encompasses the interaction between\nelectrically charged particles. It is described by Maxwell's equations, which unify electricity, magnetism,\nand light into a single theory.",
 'In essence, electric charges produce electric fields that exert forces on\nother charges, while moving charges ,currents, generate magnetic fields. These magnetic fields, in turn,\ncan affect the motion of charges and currents.',
 'The interaction between electric and magnetic fields propagates\nthrough space as electromagnetic waves, which include visible light, radio waves, and X-rays. Electromagnetic\nforces are responsible for practically all the phenomena encountered in daily life, excluding gravity.']

In [7]:
def generate_long(text, cps=14, overlap=100, output=None, speaker=None):
    global atoks, stoks
    chunks = split_and_prepare_text(text)
    speaker = pipe.default_speaker if speaker is None else speaker
    r = []
    old_stoks = None
    old_atoks = None
    for chunk in chunks:
        print(chunk)
        stoks = pipe.t2s.generate(chunk, cps=cps, show_progress_bar=False)[0]
        stoks = stoks[stoks != 512]
        if old_stoks is not None:
            assert(len(stoks) < 750-overlap)
            stoks = torch.cat([old_stoks[-overlap:], stoks])
            atoks_prompt = old_atoks[:,:,-overlap*3:]
        else:
            atoks_prompt = None
        atoks = pipe.s2a.generate(stoks, atoks_prompt=atoks_prompt, speakers=speaker.unsqueeze(0), show_progress_bar=False)
        if atoks_prompt is not None: atoks = atoks[:,:,overlap*3+1:]
        r.append(atoks)
        old_stoks = stoks
        old_atoks = atoks
        pipe.vocoder.decode_to_notebook(atoks)
    audios = []
    for i,atoks in enumerate(r):
        if i != 0: audios.append(torch.zeros((1, int(24000*0.5)), dtype=atoks.dtype, device=atoks.device))
        audios.append(pipe.vocoder.decode(atoks))
    if output:
        torchaudio.save(output, torch.cat(audios, -1).cpu(), 24000)
        from IPython.display import display, HTML, Audio
        display(HTML(f'<a href="{output}" target="_blank">Listen to {output}</a>'))

In [17]:
# generate_long(sample_text, output="test.mp3")

In [21]:
# speaker = pipe.extract_spk_emb('https://upload.wikimedia.org/wikipedia/commons/7/75/Winston_Churchill_-_Be_Ye_Men_of_Valour.ogg')
speaker = pipe.extract_spk_emb('./coach_bennet_audio.mp3')

In [22]:
generate_long(sample_text, output="test-churchill.mp3", speaker=speaker)

 It's Coach Bennett here! Listen up, champions! Here's your nugget of wisdom for today: Your mind quits a thousand times before your body does. So when that little voice says 'I'm done,' you look it straight in the eye and say 'We're just getting started!'


Remember, every step is a victory, every mile a triumph. Your legs might be screaming, but your heart? It's singing! So lace up tight, stand tall, and show that road who's boss. You're not just running, you're rewriting your story with every stride.
